# Assignment 2 - Restaurant and Consumer datasets
---

## Importing the data

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 10000)

In [2]:
data_accepts = pd.read_csv('data/chefmozaccepts.csv')
restaurant_cuisine = pd.read_csv('data/chefmozcuisine.csv')
restaurant_hours = pd.read_csv('data/chefmozhours4.csv', names=['placeID', 'hours', 'days'])
data_parking = pd.read_csv('data/chefmozparking.csv')
geoplaces = pd.read_csv('data/geoplaces2.csv', encoding='latin1')
data_rating = pd.read_csv('data/rating_final.csv')
data_user_cuisine = pd.read_csv('data/usercuisine.csv')
user_payment = pd.read_csv('data/userpayment.csv')
user_profile = pd.read_csv('data/userprofile.csv')

---
## Cleaning the data

In this section, I will be using the methods below. The fisrt one replaces missing values by Numpy's NaN. The second one computes the ratio of missing values for every column of a DataFrame. The third one replaces all the strings by their lowercase representation, so as to group related names.

In [3]:
def normalize_missing(df):
    return df.replace(to_replace=['?', 'None', 'none'], value=np.nan)

In [4]:
def columns_na_ratio(df):
    return (df.isna().sum() / len(df.index)).round(3)

In [5]:
def drop_empty_rows(df):
    print(int(0.9*len(df.columns)))
    return df.dropna(axis='index', thresh=5)

In [6]:
def to_lowercase(df):
    return df.applymap(lambda x:x.lower() if type(x) == str else x)

### Geoplaces

First of all, let's clean the geographical dataset by dropping useless columns and checking if it contains any duplicate rows.

In [7]:
geoplaces.drop(columns=['the_geom_meter', 'fax', 'zip', 'url'], inplace=True)
geoplaces = to_lowercase(geoplaces)
geoplaces = normalize_missing(geoplaces)
print(geoplaces.duplicated().sum(), "duplicated rows")

0 duplicated rows


This dataset has no duplicated rows, let's group all unavailable values and check if some columns aren't worth looking at.

In [8]:
columns_na_ratio(geoplaces)

placeID           0.000
latitude          0.000
longitude         0.000
name              0.000
address           0.208
city              0.138
state             0.138
country           0.215
alcohol           0.000
smoking_area      0.538
dress_code        0.000
accessibility     0.000
price             0.000
Rambience         0.000
franchise         0.000
area              0.000
other_services    0.915
dtype: float64

We can see that 'other services' contains more than 90% of unkwown values, so let's drop this column.

In [9]:
geoplaces.drop(columns='other_services', inplace=True)

### User profile

In [10]:
user_profile = to_lowercase(user_profile)
user_profile = normalize_missing(user_profile)
print(user_profile['userID'].duplicated().sum(), "duplicated rows")
columns_na_ratio(user_profile)

0 duplicated rows


userID              0.000
latitude            0.000
longitude           0.000
smoker              0.022
drink_level         0.000
dress_preference    0.036
ambience            0.043
transport           0.051
marital_status      0.029
hijos               0.080
birth_year          0.000
interest            0.217
personality         0.000
religion            0.217
activity            0.051
color               0.000
weight              0.000
budget              0.051
height              0.000
dtype: float64

We made sure that each row represents a different user, as there are no duplicated userID.
We can drop 'interest' and 'religion' columns as they both have 20% unknown values and we won't be using them in the questions.

In [11]:
user_profile.drop(columns=['interest', 'religion'], inplace=True)

### User payments

In [12]:
user_payment = to_lowercase(user_payment)
user_payment = normalize_missing(user_payment)
print(user_payment.duplicated().sum(), "duplicated rows")
print(user_payment['userID'].duplicated().sum(), "duplicated rows")
columns_na_ratio(user_payment)

0 duplicated rows
44 duplicated rows


userID      0.0
Upayment    0.0
dtype: float64

This DataFrame is already clean, with no missing value and no duplicated rows. It is important to note that some clients have multiple means of payment.

## Restaurant Hours

In [13]:
restaurant_hours = to_lowercase(restaurant_hours)
restaurant_hours = normalize_missing(restaurant_hours)
print(restaurant_hours.duplicated().sum(), "duplicated rows")
columns_na_ratio(restaurant_hours)

271 duplicated rows


placeID    0.0
hours      0.0
days       0.0
dtype: float64

This DataFrame has a lot of duplicated rows, let's remove them.

In [14]:
restaurant_hours.drop_duplicates(inplace=True)
print(restaurant_hours.duplicated().sum(), "duplicated rows")

0 duplicated rows


## Restaurant Cuisine

In [15]:
restaurant_cuisine = to_lowercase(restaurant_cuisine)
restaurant_cuisine = normalize_missing(restaurant_cuisine)
print(restaurant_cuisine.duplicated().sum(), "duplicated rows")
print(restaurant_cuisine['placeID'].duplicated().sum(), "duplicated restaurants")
columns_na_ratio(restaurant_cuisine)

0 duplicated rows
147 duplicated restaurants


placeID     0.0
Rcuisine    0.0
dtype: float64

There are no duplicated rows and this DataFrame is already clean. It is important to note that some restaurants offer multiple types of cuisine.

---
## Questions - Part 1

**Question 1**
What are the names of different restaurants in the state of 'tamaulipas' ?

In [16]:
geoplaces[geoplaces['state'] == 'tamaulipas']['name']

3             little pizza emilio portes gil
4                              carnitas_mata
6                         taqueria el amigo 
8                   pollo_frito_buenos_aires
19                            tacos el guero
41                    hamburguesas la perica
73                                palomo tec
76                        tacos correcaminos
86             carreton de flautas y migadas
89                        gorditas dona tota
94                             little cesarz
103    carnitas mata  calle 16 de septiembre
106                       puesto de gorditas
109    carnitas mata calle emilio portes gil
122                                tacos abi
123                    la perica hamburguesa
Name: name, dtype: object

**Question 2**
How many different customers used public transport for going to the
restaurants ?

For this question, I assume each entry in 'user profile' is someone who actually went to the restaurants, as this dataset is collected from people going to restaurants.

In [17]:
user_profile['transport'].value_counts()

public       82
car owner    35
on foot      14
Name: transport, dtype: int64

We see that 82 customers used public transport.

**Question 3** What is the least popular payment method among customers ?


In [18]:
user_payment['Upayment'].value_counts()

cash                   131
bank_debit_cards        22
visa                    17
mastercard-eurocard      4
american_express         3
Name: Upayment, dtype: int64

In [19]:
user_payment[user_payment['Upayment'] == 'american_express'].count() / user_payment.count()

userID      0.016949
Upayment    0.016949
dtype: float64

The american express is the least popular payment method, with less than 2% of payments done via this method.

**Question 4** How many (different) restaurants work until 19:00 in the evenings ?

This question is ambiguous. Does it mean restaurants working *at least* until 19:00 or restaurants working until 19:00 and then closing ? In the answer, I assume the question seeks to find restaurants closing at 19:00.

In [29]:
restaurant_hours[restaurant_hours['hours'].str.contains('19:00;')]['placeID'].nunique()

28

28 different restaurants are open until 19:00, and then close.

**Question 5** Which type of cooking practice (rcuisine) is the most common among
restaurants ?

In [21]:
restaurant_cuisine['Rcuisine'].value_counts().head()

mexican          239
international     62
american          59
dutch-belgian     55
italian           42
Name: Rcuisine, dtype: int64

In [22]:
restaurant_cuisine[restaurant_cuisine['Rcuisine'] == 'mexican'].count() / restaurant_cuisine.count()

placeID     0.260917
Rcuisine    0.260917
dtype: float64

The most common cooking practice is mexican food, with more than 25% of the restaurants serving mexican.

**Question 6** What is the percentage of customers who were born between 1980 and
1990 ?

In [23]:
(user_profile[(user_profile['birth_year'] >= 1980) & (user_profile['birth_year'] <= 1990)].count() / user_profile.count())['userID']

0.6304347826086957

63% of customers were born between 1980 and 1990 included.

**Question 7** What is the percentage of students with a medium budget preferring
walking to the restaurants ?

In [24]:
students_medium = user_profile[(user_profile['activity'] == 'student') & (user_profile['budget'] == 'medium')]
students_medium['transport'].value_counts()

public       43
car owner    18
on foot       8
Name: transport, dtype: int64

In [25]:
(students_medium[students_medium['transport'] == 'on foot'].count() / students_medium.count())['userID']

0.11267605633802817

11% of students with a medium budget prefer walking to the restaurants.

---
## Questions - Part 2

**Question 8** What are the names of restaurants that do not have public parking lots ?

**Question 9** What are the addresses of restaurants which only accept cash ?

**Question 10** Name the cities where the restaurants cook and serve ‘fast food’.

**Question 11** What are the names of restaurants that do not have public parking lots?

**Question 12** What are the names of restaurants that do not have public parking lots?

**Question 13** What are the names of restaurants that do not have public parking lots?